In [3]:
import pandas as pd


In [5]:
user_df = pd.read_csv('../CampsiteInfo - Sheet1.csv')
user_df.head()


,Date of Query,Name,Rec Area,Weeks in Advance,Number of Nights,Weekends Only,Email
0,11/30/2024,Chad Breece,Yosemite,3,1,0,cmebreece@gmail.com
1,11/30/2024,Chad Breece,Joshua Tree,3,1,0,cmebreece@gmail.com


In [6]:
uniq_users = user_df['Name'].unique()
uniq_users


array(['Chad Breece'], dtype=object)

In [7]:
for user in uniq_users:
    uniq_recs = user_df[user_df['Name'] == user]['Rec Area'].unique()
print(user, uniq_recs)


Chad Breece ['Yosemite' 'Joshua Tree']


In [ ]:
api_key = '1ed26f91-3a03-4186-be94-b3069797144e'

In [8]:
import requests
r = requests.get('https://ridb.recreation.gov/api/v1/recareas/2991?full=false')
r.json()

{'error': 'Unauthorized Access'}